In [1]:
import yfinance as yf
import pandas as pd
import warnings
import plotly.express as px

from optimisation_markowitz import MarkowitzOptimization
from optimisation_simm import SingleFactorModel, SingleFactorModelOptimization
from utils import Utils
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [2]:
start_date = "2023-04-01"
end_date = "2024-04-01"
annualized_factor = 252

We will use S&P500 Healthcare Index as benchmark to evaluate the performance with Single Index Market Model and Constant Correlation Model. The return over four years will be used as the market return in the formula

In [3]:
# Get close data for S&P 500 Information Technology Index
market_index_data = Utils.get_historical_data("^SP500-35", start_date, end_date)

# calculate simple return for market index
market_index_simple_return = Utils.calculate_simple_return(market_index_data)

# calculate daily market data returns.
# use that to calculate expected returns and variance
market_index_daily_returns = Utils.calculate_daily_returns(market_index_data)
market_index_data_expected_return = market_index_daily_returns.mean()
market_index_data_variance = market_index_daily_returns.var()

print(f"Market Index Simple Return from {start_date} - {end_date} in %: ", market_index_simple_return * 100)
print("Market Index Expected daily Return in %: ", market_index_data_expected_return * 100)
print("Market Index daily Variance in %: ", market_index_data_variance * 100)

[*********************100%%**********************]  1 of 1 completed
Market Index Simple Return from 2023-04-01 - 2024-04-01 in %:  12.896186517907182
Market Index Expected daily Return in %:  0.051107595660645815
Market Index daily Variance in %:  0.004390164722301008


For markowitz model, we will use 3-Month US Treasury Bill as risk-free rate. Since we are holding the portfolio for 1 year, we will take the last value as the risk-free rate.

In [4]:
risk_free_rate_df = yf.download("^IRX", start_date, end_date)
risk_free_rate = risk_free_rate_df['Adj Close'].iloc[-1] / 100
risk_free_rate

[*********************100%%**********************]  1 of 1 completed


0.05203000068664551

# Healthcare Sector ETFs

We will now download the data for US Healthcare Equity ETFs on which we will run our strategies. We will use the following top 5 ETFs based on total assets:
1. Healthcare Select Sector SPDR Fund (XLV)
2. Vanguard Health Care ETF (VHT)
3. iShares Biotechnology ETF (IBB)
4. SPDR S&P Biotech ETF (XBI)
5. iShares U.S. Medical Devices ETF (IHI)

Source: [https://etfdb.com/etfdb-category/technology-equities/ ](https://etfdb.com/etfdb-category/health-biotech-equities/)

In [6]:
# get the data
xlv_data = Utils.get_historical_data("XLV", start_date, end_date)
vht_data = Utils.get_historical_data("VHT", start_date, end_date)
ibb_data = Utils.get_historical_data("IBB", start_date, end_date)
xbi_data = Utils.get_historical_data("XBI", start_date, end_date)
ihi_data = Utils.get_historical_data("IHI", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [7]:
# check for any missing data
print(market_index_data.isnull().sum())
print(xlv_data.isnull().sum())
print(vht_data.isnull().sum())
print(ibb_data.isnull().sum())
print(xbi_data.isnull().sum())
print(ihi_data.isnull().sum())


Adj Close        0
simple_return    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64


In [8]:

xlv_simple_return = Utils.calculate_simple_return(xlv_data)
vht_simple_return = Utils.calculate_simple_return(vht_data)
ibb_simple_return = Utils.calculate_simple_return(ibb_data)
xbi_simple_return = Utils.calculate_simple_return(xbi_data)
ihi_simple_return = Utils.calculate_simple_return(ihi_data)

simple_returns_dict = {
    "XLV": xlv_simple_return,
    "VHT": vht_simple_return,
    "IBB": ibb_simple_return,
    "XBI": xbi_simple_return,
    "IHI": ihi_simple_return
}

for key, value in simple_returns_dict.items():
    print(f"{key} Simple Return from {start_date} - {end_date} in %: ", value * 100)

XLV Simple Return from 2023-04-01 - 2024-04-01 in %:  14.699930196487564
VHT Simple Return from 2023-04-01 - 2024-04-01 in %:  13.941585714239846
IBB Simple Return from 2023-04-01 - 2024-04-01 in %:  6.075116306528638
XBI Simple Return from 2023-04-01 - 2024-04-01 in %:  23.162156584619375
IHI Simple Return from 2023-04-01 - 2024-04-01 in %:  9.752411724222942


In [9]:
# plot the simple retruns using plotly for all stocks along with market index
fig = px.line(title="Simple Returns for HealthCare Stocks and Market Index")
fig.add_scatter(x=xlv_data.index, y=xlv_data['simple_return'], mode='lines', name='XLV')
fig.add_scatter(x=vht_data.index, y=vht_data['simple_return'], mode='lines', name='VHT')
fig.add_scatter(x=ibb_data.index, y=ibb_data['simple_return'], mode='lines', name='IBB')
fig.add_scatter(x=xbi_data.index, y=xbi_data['simple_return'], mode='lines', name='XBI')
fig.add_scatter(x=ihi_data.index, y=ihi_data['simple_return'], mode='lines', name='IHI')
fig.add_scatter(x=market_index_data.index, y=market_index_data['simple_return'], mode='lines', name='Market Index')
fig.show()

On observing the simple returns, we see that SPDR S&P Biotech ETF (XBI) had the best performance, even outperforming the market. This ETF consists of many small-cap biotech companies, which are known for their high volatility providing it as a speculative investment which is justified by the sharper U-Shape returns curve.

In [10]:
xlv_daily_returns = Utils.calculate_daily_returns(xlv_data)
vht_daily_returns = Utils.calculate_daily_returns(vht_data)
ibb_daily_returns = Utils.calculate_daily_returns(ibb_data)
xbi_daily_returns = Utils.calculate_daily_returns(xbi_data)
ihi_daily_returns = Utils.calculate_daily_returns(ihi_data)

daily_returns_df = pd.DataFrame([xlv_daily_returns, vht_daily_returns, ibb_daily_returns, xbi_daily_returns, ihi_daily_returns]).T
daily_returns_df.columns = ['XLV', 'VHT', 'IBB', 'XBI', 'IHI']

In [11]:
# we can now plot the daily returns for all the stocks in sub plots
fig = make_subplots(rows=3, cols=2, subplot_titles=("XLV", "VHT", "IBB", "XBI", "IHI", "Market Index"))

fig.add_scatter(x=xlv_data.index, y=xlv_daily_returns, mode='lines', name='XLV', row=1, col=1)
fig.add_scatter(x=vht_data.index, y=vht_daily_returns, mode='lines', name='VHT', row=1, col=2)
fig.add_scatter(x=ibb_data.index, y=ibb_daily_returns, mode='lines', name='IBB', row=2, col=1)
fig.add_scatter(x=xbi_data.index, y=xbi_daily_returns, mode='lines', name='XBI', row=2, col=2)
fig.add_scatter(x=ihi_data.index, y=ihi_daily_returns, mode='lines', name='IHI', row=3, col=1)

fig.add_scatter(x=market_index_data.index, y=market_index_daily_returns, mode='lines', name='Market Index', row=3, col=2)

fig.update_layout(title_text="Daily Returns for Healthcare Stocks and Market Index", height=1000, width=1000)
fig.show()


In [12]:
cov_matrix = daily_returns_df.cov()
cov_matrix

,XLV,VHT,IBB,XBI,IHI
XLV,0.000044,0.000044,0.000047,0.000055,0.000045
VHT,0.000044,0.000046,0.000055,0.000072,0.000049
IBB,0.000047,0.000055,0.000106,0.000162,0.000058
XBI,0.000055,0.000072,0.000162,0.000325,0.000085
IHI,0.000045,0.000049,0.000058,0.000085,0.000101


In [13]:
corr_matrix = daily_returns_df.corr()
corr_matrix

,XLV,VHT,IBB,XBI,IHI
XLV,1.000000,0.983161,0.691485,0.459904,0.682623
VHT,0.983161,1.000000,0.780824,0.586944,0.724791
IBB,0.691485,0.780824,1.000000,0.873231,0.563853
XBI,0.459904,0.586944,0.873231,1.000000,0.470927
IHI,0.682623,0.724791,0.563853,0.470927,1.000000


Unlike Technology ETFs, HealthCare Sector ETFs have a large variation in correlation ranging from 0.45 - 0.97.

In [14]:
# standard deviations between daily returns
std_dev = daily_returns_df.std()
std_dev

XLV    0.006632
VHT    0.006796
IBB    0.010294
XBI    0.018034
IHI    0.010032
dtype: float64

In [15]:
# daily mean return
mean_daily_return = daily_returns_df.mean()
mean_daily_return

XLV    0.000575
VHT    0.000549
IBB    0.000291
XBI    0.001002
IHI    0.000426
dtype: float64

## Optimal tangency portfolio using Markowitz model 

In [16]:
# Run the Markowitz Portfolio Optimization
simple_returns_list = list(simple_returns_dict.values())
markowitz = MarkowitzOptimization(simple_returns_list, cov_matrix, risk_free_rate)


In [17]:
# find the weights of the portfolio with short shelling
markowitz_weights_with_ss = markowitz.find_tangency_portfolio_with_short_selling()

In [32]:
# create the wieghts dictionary
markowitz_weights_with_ss_dict = {
    "XLV": markowitz_weights_with_ss[0],
    "VHT": markowitz_weights_with_ss[1],
    "IBB": markowitz_weights_with_ss[2],
    "XBI": markowitz_weights_with_ss[3],
    "IHI": markowitz_weights_with_ss[4]
}

for ticker, weight in markowitz_weights_with_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights with Short Selling in %: ", weight)

XLV -> Markowitz Weights with Short Selling in %:  11.929457713665768
VHT -> Markowitz Weights with Short Selling in %:  -8.332737012207474
IBB -> Markowitz Weights with Short Selling in %:  -4.6554410817532395
XBI -> Markowitz Weights with Short Selling in %:  2.6035304876832934
IHI -> Markowitz Weights with Short Selling in %:  -0.5448101073883471


In [19]:
# find the weights of the portfolio without short shelling
markowitz_weights_without_ss = markowitz.find_tangency_portfolio_no_short_selling()

In [33]:
# weights dictionary
markowitz_weights_without_ss_dict = {
    "XLV": markowitz_weights_without_ss[0],
    "VHT": markowitz_weights_without_ss[1],
    "IBB": markowitz_weights_without_ss[2],
    "XBI": markowitz_weights_without_ss[3],
    "IHI": markowitz_weights_without_ss[4]
}

for ticker, weight in markowitz_weights_without_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling in %: ", weight * 100)

XLV -> Markowitz Weights without Short Selling in %:  88.69463524664923
VHT -> Markowitz Weights without Short Selling in %:  3.8993962139105554e-10
IBB -> Markowitz Weights without Short Selling in %:  6.401055033244951e-10
XBI -> Markowitz Weights without Short Selling in %:  11.30536475180763
IHI -> Markowitz Weights without Short Selling in %:  5.131018006310217e-10


In [26]:
volatility_with_ss = Utils.calculate_markowitz_portfolio_risk(markowitz_weights_with_ss, cov_matrix)
print("Volatility with Short Selling in %: ", volatility_with_ss * 100)

Volatility with Short Selling in %:  2.144663766941611


In [27]:
portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling in %: ", portfolio_mean_return_with_ss * 100)

Mean Portfolio Return with Short Selling in %:  5.246960141749889


In [28]:
# calculate sharpe ratio
sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss, volatility_with_ss, risk_free_rate)
print("Sharpe Ratio with Short Selling: ", sharpe_ratio_with_ss)

Sharpe Ratio with Short Selling:  0.5962514575950296


In [29]:
volatility_without_ss = Utils.calculate_markowitz_portfolio_risk(markowitz_weights_without_ss, cov_matrix)
print("Volatility without Short Selling in %: ", volatility_without_ss * 100)

Volatility without Short Selling in %:  0.7055785965621794


In [30]:
portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return without Short Selling in %: ", portfolio_mean_return_without_ss * 100)

Mean Portfolio Return without Short Selling in %:  0.9894370739257072


In [31]:
# calculate sharpe ratio
sharpe_ratio_without_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_without_ss, volatility_without_ss, risk_free_rate)
print("Sharpe Ratio without Short Selling: ", sharpe_ratio_without_ss)

Sharpe Ratio without Short Selling:  0.054556692988163936


From the results above, we observe that Healthcare sector benefits from short-selling as returns without short selling are around 5 times.

We also observe that without weights constraint in optimization without short selling, XLV and XBI account for around 99% proportion in the portfolio. To promote diversification , we will add a constraint that allows a maximum of 30% in each ETF

In [52]:
# we are interested in finding optimal weights when an asset can hold 30% of the portfolio max
markowitz_weights_without_ss_pct_constraint = markowitz.find_tangency_portfolio_no_short_selling_and_max_weight(0.35)

In [53]:
markowitz_weights_dict_without_ss_pct_constraint = {
    "XLV": markowitz_weights_without_ss_pct_constraint[0],
    "VHT": markowitz_weights_without_ss_pct_constraint[1],
    "IBB": markowitz_weights_without_ss_pct_constraint[2],
    "XBI": markowitz_weights_without_ss_pct_constraint[3],
    "IHI": markowitz_weights_without_ss_pct_constraint[4]
}

for ticker, weight in markowitz_weights_dict_without_ss_pct_constraint.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling and 30% constraint in %: ", weight * 100)

XLV -> Markowitz Weights without Short Selling and 30% constraint in %:  34.99999999957792
VHT -> Markowitz Weights without Short Selling and 30% constraint in %:  34.99999999968784
IBB -> Markowitz Weights without Short Selling and 30% constraint in %:  5.039377001487181e-10
XBI -> Markowitz Weights without Short Selling and 30% constraint in %:  29.99999999993722
IHI -> Markowitz Weights without Short Selling and 30% constraint in %:  2.930824832536338e-10


In [54]:
volatility_without_ss_with_constraint = Utils.calculate_markowitz_portfolio_risk(markowitz_weights_without_ss_pct_constraint, cov_matrix)
print("Volatility with Short Selling and Constraint in %: ", volatility_without_ss_with_constraint * 100)

Volatility with Short Selling and Constraint in %:  0.8821884767104925


In [55]:
portfolio_mean_return_with_ss_with_constraint = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss_pct_constraint, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling and Constraint in %: ", portfolio_mean_return_with_ss_with_constraint * 100)

Mean Portfolio Return with Short Selling and Constraint in %:  1.1017772559352943


In [56]:
# find the Sharpe Ratio
sharpe_no_ss_with_constraint = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss_with_constraint, volatility_without_ss_with_constraint, risk_free_rate)
print("Sharpe Ratio with Short Selling and Constraint: ", sharpe_no_ss_with_constraint)

Sharpe Ratio with Short Selling and Constraint:  0.04220426221927062


## Single Index Market Model

We will now compare results from markowitz model with single index market model. We will use S&P500 Healthcare Index as benchmark to evaluate the performance with Single Index Market Model.

From this point, we will run `SingleFactorModel` class over all ETFs calculating beta, alpha and tau. We will then calculate the expected return using the formula:

$$
u_i = \alpha_i + \beta_i \cdot u_m

In [57]:
def get_simm_params(asset_return: pd.DataFrame, market_return: pd.DataFrame):
  model = SingleFactorModel(asset_return, market_return)
  model.fit()
  
  params = model.get_params()
  alpha = params.get("alpha").get("value")
  beta = params.get("beta").get("value")
  tau = params.get("tau").get("value")
  
  return (alpha, beta, tau)
  

In [58]:
# run SIMM for params
xlv_alpha, xlv_beta, xlv_tau = get_simm_params(xlv_daily_returns, market_index_daily_returns)
vht_alpha, vht_beta, vht_tau = get_simm_params(vht_daily_returns, market_index_daily_returns)
ibb_alpha, ibb_beta, ibb_tau = get_simm_params(ibb_daily_returns, market_index_daily_returns)
xbi_alpha, xbi_beta, xbi_tau = get_simm_params(xbi_daily_returns, market_index_daily_returns)
ihi_alpha, ihi_beta, ihi_tau = get_simm_params(ihi_daily_returns, market_index_daily_returns)


In [59]:
# get expected returns using alpha, beta and expected market return
xlv_expected_return = xlv_alpha + (xlv_beta * market_index_data_expected_return)
vht_expected_return = vht_alpha + (vht_beta * market_index_data_expected_return)
ibb_expected_return = ibb_alpha + (ibb_beta * market_index_data_expected_return)
xbi_expected_return = xbi_alpha + (xbi_beta * market_index_data_expected_return)
ihi_expected_return = ihi_alpha + (ihi_beta * market_index_data_expected_return)

In [60]:
# we can now pack the expected returns into a dictionary along with beta and tau
expected_returns_dict = {
    "XLV": xlv_expected_return,
    "VHT": vht_expected_return,
    "IBB": ibb_expected_return,
    "XBI": xbi_expected_return,
    "IHI": ihi_expected_return
}

alpha_dict = {
    "XLV": xlv_alpha,
    "VHT": vht_alpha,
    "IBB": ibb_alpha,
    "XBI": xbi_alpha,
    "IHI": ihi_alpha
}

beta_dict = {
    "XLV": xlv_beta,
    "VHT": vht_beta,
    "IBB": ibb_beta,
    "XBI": xbi_beta,
    "IHI": ihi_beta
}

tau_dict = {
    "XLV": xlv_tau,
    "VHT": vht_tau,
    "IBB": ibb_tau,
    "XBI": xbi_tau,
    "IHI": ihi_tau
}

for i in range(len(expected_returns_dict.keys())):
  print(f"{ list(expected_returns_dict.keys())[i] } -> Expected Daily Return in %: ", list(expected_returns_dict.values())[i] * 100)
  print(f"{ list(alpha_dict.keys())[i] } -> Alpha: ", list(alpha_dict.values())[i])
  print(f"{ list(beta_dict.keys())[i] } -> Beta: ", list(beta_dict.values())[i])
  print(f"{ list(tau_dict.keys())[i] } -> Tau: ", list(tau_dict.values())[i])
  print()

XLV -> Expected Daily Return in %:  0.05750597247127688
XLV -> Alpha:  6.427081883051211e-05
XLV -> Beta:  0.9994383403866081
XLV -> Tau:  0.00035619619952188396

VHT -> Expected Daily Return in %:  0.054939990008069824
VHT -> Alpha:  3.43663596222627e-05
VHT -> Beta:  1.0077436314520756
VHT -> Tau:  0.0012654183965484093

IBB -> Expected Daily Return in %:  0.029059058391799796
IBB -> Alpha:  -0.0002556869262511136
IBB -> Beta:  1.068877342218154
IBB -> Tau:  0.00748632143957253

XBI -> Expected Daily Return in %:  0.10016452150430182
XBI -> Alpha:  0.0003682024699311224
XBI -> Beta:  1.2394297499689721
XBI -> Tau:  0.016088290736211354

IHI -> Expected Daily Return in %:  0.042557597313138275
IHI -> Alpha:  -9.906280423495968e-05
IHI -> Beta:  1.026537778943743
IHI -> Tau:  0.007389708685352987



In [61]:
# run the SIMM optimization
expected_returns_list = list(expected_returns_dict.values())
beta_list = list(beta_dict.values())
tau_list = list(tau_dict.values())
alpha_list = list(alpha_dict.values())

simm_optimization = SingleFactorModelOptimization(expected_returns_list, beta_list, tau_list, market_index_data_variance, risk_free_rate)

In [62]:
# find the optimal weights when short selling is allowed
simm_weights_with_ss = simm_optimization.find_weights_short_selling_allowed()

In [63]:
simm_weights_with_ss_dict = {
  "XLV": simm_weights_with_ss[0],
  "VHT": simm_weights_with_ss[1],
  "IBB": simm_weights_with_ss[2],
  "XBI": simm_weights_with_ss[3],
  "IHI": simm_weights_with_ss[4]
}

for ticker, weight in simm_weights_with_ss_dict.items():
    print(f"{ticker} -> SIMM Weights with Short Selling in %: ", weight * 100)

XLV -> SIMM Weights with Short Selling in %:  121.80227437824665
VHT -> SIMM Weights with Short Selling in %:  -11.427105269281455
IBB -> SIMM Weights with Short Selling in %:  -4.654185348097642
XBI -> SIMM Weights with Short Selling in %:  -4.088755642640715
IHI -> SIMM Weights with Short Selling in %:  -1.6322281182268306


With weights found, we can find the expected return of portfolio and the risk using the formula:

$$
u_p = \alpha_p + \beta_p \cdot u_m

\newline

\sigma_p^2 = \beta_p^2 \cdot \sigma_m^2 + \tau_p^2

\newline

where:
\newline

\alpha_p = \sum_{i=1}^{n} x_i \cdot \alpha_i

\newline

\beta_p = \sum_{i=1}^{n} x_i \cdot \beta_i

\newline

\tau_p^2 = \sum_{i=1}^{n} x_i^2 \cdot \tau_i^2

In [64]:
simm_portfolio_risk_with_ss = Utils.calculate_factor_model_percentage_portfolio_risk(simm_weights_with_ss, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk with Short Selling in %: ", simm_portfolio_risk_with_ss * 100)

Portfolio Risk with Short Selling in %:  0.6585779745619309


In [65]:
simm_portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(simm_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling in %: ", simm_portfolio_mean_return_with_ss * 100)

Mean Portfolio Return with Short Selling in %:  0.9147359706386662


In [66]:
# find the sharpe ratio
simm_sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_with_ss, simm_portfolio_risk_with_ss, risk_free_rate)
print("SIMM Sharpe Ratio with Short Selling: ", simm_sharpe_ratio_with_ss)

SIMM Sharpe Ratio with Short Selling:  0.04923274893257556


In [67]:
# find optimal weights when short selling is not allowed
simm_weights_no_ss = simm_optimization.find_weights_no_short_selling()

In [69]:
simm_weights_no_ss_dict = {
  "XLV": simm_weights_no_ss[0],
  "VHT": simm_weights_no_ss[1],
  "IBB": simm_weights_no_ss[2],
  "XBI": simm_weights_no_ss[3],
  "IHI": simm_weights_no_ss[4]
}

for ticker, weight in simm_weights_no_ss_dict.items():
  print(f"{ticker} -> SIMM Weights without Short Selling in %: ", weight * 100)
    

XLV -> SIMM Weights without Short Selling in %:  92.13977474022762
VHT -> SIMM Weights without Short Selling in %:  7.361250985218143
IBB -> SIMM Weights without Short Selling in %:  0.2230805849113384
XBI -> SIMM Weights without Short Selling in %:  0.05601129715082885
IHI -> SIMM Weights without Short Selling in %:  0.21988239249204314


In [70]:
# we will now work to find the expected return and risk of the portfolio
simm_portfolio_risk_no_ss = Utils.calculate_factor_model_percentage_portfolio_risk(simm_weights_no_ss, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk without Short Selling in %: ", simm_portfolio_risk_no_ss * 100)

Portfolio Risk without Short Selling in %:  0.6637293904797943


In [71]:
simm_portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(simm_weights_no_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return without Short Selling in %: ", simm_portfolio_mean_return_without_ss * 100)

Mean Portfolio Return without Short Selling in %:  0.9087306353250462


In [72]:
# find the sharpe ratio
simm_sharpe_ratio_no_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_without_ss, simm_portfolio_risk_no_ss, risk_free_rate)
print("SIMM Sharpe Ratio without Short Selling: ", simm_sharpe_ratio_no_ss)

SIMM Sharpe Ratio without Short Selling:  0.04708845696942997
